### Import Libraries

In [1]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

import threading
import time
from utils import preprocess
import torch.nn.functional as F

### Camera

In [ ]:
from jetcam.csi_camera import CSICamera

# CSIカメラの設定、幅と高さも設定している
camera = CSICamera(width=224, height=224)

# カメラを動作させる設定
camera.running = True

### Task

In [3]:
import torchvision.transforms as transforms
from xy_dataset import XYDataset

TASK = 'road_following'

CATEGORIES = ['apex']

DATASETS = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

# データAugmentataionの設定,ColorJitterはに対して明るさ、コントラスト、彩度（鮮やかさ）、色相を設定、 NormalizeはRGBに対して平均と分散で正規化を行っている
TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = XYDataset(TASK + '_' + name, CATEGORIES, TRANSFORMS, random_hflip=True)

### Data Collection

In [4]:
import cv2
import ipywidgets
import traitlets
from IPython.display import display
from jupyter_clickable_image_widget import ClickableImageWidget


# 設定したいカテゴリのデータセットを指定
dataset = datasets[DATASETS[1]]

# このセルの2度目の実行に備えてカメラからの観測をオフにする
camera.unobserve_all()

# イメージのプレビューを作成
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# データセット、カテゴリ、データの数を把握できるようにwidgetを作成
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')

# 現在のデータセットの数を取得して更新
count_widget.value = dataset.get_count(category_widget.value)

# データセットを設定
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# 新しいカテゴリのデータセットになったときのデータセットの数を更新
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

# 画像をクリックしてデータを保存。データセット名にx,y座標を加えて保存する。保存したデータを表示してカウントを更新する
def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # ディスクにデータを保存する
        dataset.save_entry(category_widget.value, camera.value, x, y)
        
        # 保存したデータを表示
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = dataset.get_count(category_widget.value)
        
camera_widget.on_msg(save_snapshot)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]),
    dataset_widget,
    category_widget,
    count_widget
])

display(data_collection_widget)

### Model

In [5]:
import torch
import torchvision

# GPUで学習するための設定
device = torch.device('cuda')
# x,y座標 * カテゴリ数
output_dim = 2 * len(dataset.categories)

# ALEXNET
# model = torchvision.models.alexnet(pretrained=True)
# model.classifier[-1] = torch.nn.Linear(4096, output_dim)

# SQUEEZENET 
# model = torchvision.models.squeezenet1_1(pretrained=True)
# model.classifier[1] = torch.nn.Conv2d(512, output_dim, kernel_size=1)
# model.num_classes = len(dataset.categories)

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)

# RESNET 34
# model = torchvision.models.resnet34(pretrained=True)
# model.fc = torch.nn.Linear(512, output_dim)

# DENSENET 121
# model = torchvision.models.densenet121(pretrained=True)
# model.classifier = torch.nn.Linear(model.num_features, output_dim)

# GPU か CPUの設定
model = model.to(device)

# モデルのパスの設定と保存、ロードするためのボタンを表示
model_save_button = ipywidgets.Button(description='save model')
model_load_button = ipywidgets.Button(description='load model')
model_path_widget = ipywidgets.Text(description='model path', value='road_following_model_A.pth')

# pytorchのモデルをロードするための関数
def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))

# モデルをロードする機能を追加
model_load_button.on_click(load_model)
    
# モデルを保存する関数
def save_model(c):
    torch.save(model.state_dict(), model_path_widget.value)

# モデルを保存する機能を追加
model_save_button.on_click(save_model)

# ボタンを設定
model_widget = ipywidgets.VBox([
    model_path_widget,
    ipywidgets.HBox([model_load_button, model_save_button])
])


display(model_widget)

### Live Execution

In [6]:
# カメラ機能の状態を設定するためのボタン
state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
# 予測の様子を表示するためのWidget
prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

# 予測の状況をライブする関数
def live(state_widget, model, camera, prediction_widget):
    global dataset
    # live状態であると動作
    while state_widget.value == 'live':
        # カメラからデータを取得
        image = camera.value
        # 前処理を行う
        preprocessed = preprocess(image)
        # 予測を行い、予測値をCPUで処理する形に変更
        output = model(preprocessed).detach().cpu().numpy().flatten()
        # 設定したカテゴリ値を取得
        category_index = dataset.categories.index(category_widget.value)
        # x, y 座標に変換
        x = output[2 * category_index]
        y = output[2 * category_index + 1]
        
        # カメラの座標に変換。中央からの値にしたいため、2で割って0.5を足している
        x = int(camera.width * (x / 2.0 + 0.5))
        y = int(camera.height * (y / 2.0 + 0.5))
        
        # カメラ画像をコピー
        prediction = image.copy()
        # カメラの予測値をサークルとして追加
        prediction = cv2.circle(prediction, (x, y), 8, (255, 0, 0), 3)
        # カメラの予測値をjpeg値に変換してwidgetに追加
        prediction_widget.value = bgr8_to_jpeg(prediction)

# カメラの状態を確認してカメラを起動
def start_live(change):
    if change['new'] == 'live':
        execute_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget))
        execute_thread.start()

# カメラ機能の状態を確認してカメラを起動するかどうかを設定
state_widget.observe(start_live, names='value')

# カメラの状態を設定するボタンと予測の状態を表示する画面を設定
live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])

display(live_execution_widget)

### Training

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

# RGBを正規化するための平均と分散を設定
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()


def preprocess(image):
    image = PIL.Image.fromarray(image)
    # FP16の精度でPyTorchで扱えるデータ形式（Tensor）で変換し、さらにGPUで予測可能なデータに変換
    image = transforms.functional.to_tensor(image).to(device).half()
    # 平均と分散で正規化
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [7]:
BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

# Epoch, 評価、学習、lossをボタンとして表示、学習の進行状況を確認するプログレスバーを追加
epochs_widget = ipywidgets.IntText(description='epochs', value=1)
eval_button = ipywidgets.Button(description='evaluate')
train_button = ipywidgets.Button(description='train')
loss_widget = ipywidgets.FloatText(description='loss')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')

# 学習用のデータセットを指定。indexが0の場合はAのデータセット, indexが1の場合はBのデータセット, 
dataset = datasets[DATASETS[1]]

def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget
    
    try:
        # 学習データをローダーとして設定
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        # 学習中にボタン操作しないための設定
        state_widget.value = 'stop'
        train_button.disabled = True
        eval_button.disabled = True
        time.sleep(1)

        # モデルの学習と評価で動作を振り分け
        if is_training:
            model = model.train()
        else:
            model = model.eval()

        # Epochの間、学習を行う
        while epochs_widget.value > 0:
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            # バッチサイズごとに学習データ、カテゴリ、xy座標を取得
            for images, category_idx, xy in iter(train_loader):
                # GPU用にデータを変換
                images = images.to(device)
                preprocessed = preprocess(images)
                xy = xy.to(device)

                if is_training:
                    # 勾配を初期化して前回の勾配を使用しないように
                    optimizer.zero_grad()

                # モデルの出力を取得
                outputs = model(images)

                # カテゴリに応じたx,y座標に対してMSEロスを計算
                loss = 0.0
                for batch_idx, cat_idx in enumerate(list(category_idx.flatten())):
                    loss += torch.mean((outputs[batch_idx][2 * cat_idx:2 * cat_idx+2] - xy[batch_idx])**2)
                loss /= len(category_idx)

                if is_training:
                    # 誤差逆伝番を実行
                    loss.backward()

                    # パラメータを調整
                    optimizer.step()

                # プログレスバーの更新
                count = len(category_idx.flatten())
                i += count
                progress_widget.value = i / len(dataset)
                # トータルロスの更新
                sum_loss += float(loss)
                loss_widget.value = sum_loss / i
                
            # 学習時はepochを現象させる。評価時は1回のみの実行で終了
            if is_training:
                epochs_widget.value = epochs_widget.value - 1
            else:
                break
    except e:
        print("Error {}".format(e))
        pass
    # モデルが評価用になる。Batch Normが学習時の平均、分散を使用。Drop Outは働かなくなる
    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False
    state_widget.value = 'live'

# 学習と評価用のボタンを追加
train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))

# epoch, プログレスバー、ロス、学習、評価用ボタンをまとめて扱う
train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    ipywidgets.HBox([train_button, eval_button])
])

# Widgetを表示
display(train_eval_widget)

# TensorRT Convert model

In [8]:
import torch
import torchvision

CATEGORIES = ['apex']

# # GPU用にデバイスを設定
# device = torch.device('cuda')
# # モデルの型だけを取得
# model = torchvision.models.resnet18(pretrained=False)
# # モデルの最終層を変更（カテゴリ*xy座標）
# model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
# # モデルをGPUの評価用に変更
# model = model.cuda().eval().half()
model = model.cuda().eval()

Next, load the saved model.  Enter the model path you used to save.

In [9]:
# モデルをロードして先ほど設定したモデルの型に重みを設定
# model.load_state_dict(torch.load('road_following_model2.pth'))

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [10]:
%%time
from torch2trt import torch2trt

# 空のデータをTensorRT変換用に準備
# TensorRT
#    https://developer.nvidia.com/tensorrt
data = torch.zeros((1, 3, 224, 224)).cuda()

# モデルをFP16でTensorRT用に変更
model_trt = torch2trt(model, [data], fp16_mode=True)

CPU times: user 12.6 s, sys: 6.74 s, total: 19.4 s
Wall time: 37 s


Save the optimized model using the cell below

In [11]:
# モデルを保存
torch.save(model_trt.state_dict(), 'road_following_model_B2_trt.pth')

In [ ]:
camera.unobserve_all()